In [1]:
import openalea.plantconvert as pc
import openalea.plantgl.all as pgl
import openalea.strawberry as strawberry
from openalea.mtg.algo import orders
from openalea.strawberry import import_mtgfile, visu3d

In [2]:
DATA = "/".join(strawberry.__file__.split("/")[:-3]) + "/share/data"
CAPRISS = DATA + "/Capriss.mtg"

In [3]:
gariguette = import_mtgfile.import_mtgfile(filename=["Gariguette"])
gariguette.properties()["order"] = orders(gariguette)
gariguette.roots(scale=1)
print(gariguette)

MTG : nb_vertices=6519, nb_scales=4


In [4]:
%gui qt
scene = visu3d.plot3d(gariguette, by=["Sample_date"], hide_leaves=False, display=False)
#pgl.Viewer.display(scene)

In [5]:
def deep_primitive(sh):
    if isinstance(sh, pgl.Shape):
        return deep_primitive(sh.geometry)
    elif isinstance(sh, pgl.Transformed):
        return deep_primitive(sh.geometry)
    else:
        return sh
    
def merge_shapes(shapes):
    if len(shapes) > 1:
        color = shapes[0].appearance
        geo = pgl.tesselate(pgl.Group(*[sh.geometry for sh in shapes]))
        if not isinstance(geo, pgl.TriangleSet):
            print(geo)
        return pgl.Shape(geo, color)
    else:
        # if there is only one shape in the list, we don't need to merge
        return shapes[0]

def remove_bezier(scene_dict):
    keys = list(scene_dict.keys())
    shapes = list(scene_dict.values())
    poped_nb = 0
    for i, sh in enumerate(scene_dict.values()):
        if isinstance(deep_primitive(sh), pgl.BezierCurve2D):
            keys.pop(i - poped_nb)
            shapes.pop(i - poped_nb)
            poped_nb += 1
    return dict(zip(keys, shapes))

In [6]:
scene_dict = scene.todict()
keys = iter(scene_dict.keys())
merged = (merge_shapes(shapes) for shapes in scene_dict.values())
scene_dict_merged = dict(zip(keys, merged))
scene_dict_merged = remove_bezier(scene_dict_merged)
pc.opf.writer.apply_scene(gariguette, scene_dict_merged)
plant_gariguette = pc.plantconvert.Plant(ignored_attrs=["index", "order", "color"])
plant_gariguette.mtg = gariguette

In [7]:
plant_gariguette.write("gariguette.opf")

NameError: name '_get_attribute_types' is not defined

In [9]:
plant_gariguette.write("gariguette.vtp")

StopIteration: 

In [14]:
types = []
for names in plant_gariguette.mtg.property_names():
    val = iter(plant_gariguette.mtg.property(names).values())
    print(val)

In [25]:
isinstance(list(plant_gariguette.mtg.property(plant_gariguette.mtg.property_names()[0]).values())[0], str)

True

In [17]:
plant_gariguette.mtg.property_names()[0]

'edge_type'

In [18]:
plant_gariguette.mtg.property_names()[0].values()

AttributeError: 'str' object has no attribute 'values'

In [20]:
plant_gariguette.mtg.property()

TypeError: property() missing 1 required positional argument: 'name'